## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

'https://drive.google.com/file/d/1mG3tPS_59pANrgwd6T2IgnHWgph4vYbg/view?usp=sharing'

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)

In [1]:
!pip install sentencepiece
!pip install pymorphy2

     |████████████████████████████████| 1.1MB 4.4MB/s 
     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 8.2MB 7.7MB/s 


In [2]:
import re
import sentencepiece as spm

import smart_open as sm
import gensim
import numpy as np
import tqdm

import pymorphy2


In [3]:
morph = pymorphy2.MorphAnalyzer()

In [4]:
def read_data(f_in):
    with sm.open(f_in, encoding='utf-8') as f:
        for line in f:
            category, headline, text = line.strip().split('\t')
            yield category, headline, text
            
        
def normalize_text(text):
    text = text.lower()
    words = re.findall(r'\b\w+\b', text.lower())
    
    return words


def prepare_spm_file(f_in):
    data =[]
    for category, headline, text in read_data(f_in):
        
        category = morph.parse(category)[0].normal_form
        headline = [morph.parse(word)[0].normal_form for word in normalize_text(headline)]
        text = [normalize_text(sent) for sent in re.split(r'[.!?]', text) if len(sent) > 50]
        text_2 =[]
        
        for j in range(len(text)):
            text_2.append([morph.parse(word)[0].normal_form for word in text[j]])
        data.append([category, headline, text_2])
       
    return data

In [29]:
data_train = prepare_spm_file('./news_train.txt')
data_test = prepare_spm_file('./news_test.txt')

In [30]:
print(data_train[0])
print(data_test[0])

['sport', ['овечкин', 'пожертвовать', 'детский', 'хоккейный', 'школа', 'автомобиль'], [['нападать', 'вашингтон', 'кэпиталзти', 'александр', 'овечкин', 'передать', 'детский', 'хоккейный', 'школа', 'автомобиль', 'получить', 'они', 'после', 'окончание', 'матч', 'весь', 'звезда', 'национальный', 'хоккейный', 'лига', 'нхл'], ['автомобиль', 'honda', 'accord', 'быть', 'подарить', 'хоккеист', 'по', 'решение', 'спонсор', 'мероприятие'], ['игрок', 'нхл', 'пожертвовать', 'машина', 'спортивный', 'школа', 'nova', 'cool', 'cats', 'special', 'hockey', 'inc'], ['овечкин', 'общаться', 'с', '10', 'летний', 'девочка', 'анна', 'чтоб', 'с', 'синдром', 'даун', 'который', 'заниматься', 'в', 'этот', 'школа', 'и', 'являться', 'поклонница', 'спортсмен'], ['в', 'сентябрь', 'форвард', 'пообедать', 'вместе', 'с', 'юный', 'хоккеистка', 'в', 'японский', 'ресторан'], ['матч', 'весь', 'звезда', 'нхл', 'в', 'коламбус', 'штат', 'огайо', 'завершиться', 'победа', 'команда', 'джонатан', 'тэйвз', 'над', 'команда', 'ник', 'ф

2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать gensim . Продемонстрировать семантические ассоциации.

In [7]:
from gensim.models import Word2Vec

In [32]:
sentences = [word[1] for word in data_train]

for sentence in data_train:
    for i in range(len(sentence[2])):
        sentences.extend([sentence[2][i]])

w2v = Word2Vec(sentences, workers=8)

In [35]:
print(w2v.wv.most_similar(positive=["футбол"]))
print(w2v.wv.most_similar(positive=["искусство"]))

[('хоккей', 0.9226015210151672), ('теннис', 0.8386198282241821), ('биатлон', 0.7951208353042603), ('фигурный', 0.7372260093688965), ('уефа', 0.7363604307174683), ('шахматы', 0.7299032807350159), ('баскетбол', 0.7280426025390625), ('кхл', 0.7242833375930786), ('чемпионка', 0.7230687141418457), ('кубок', 0.722800612449646)]
[('театральный', 0.8301874399185181), ('литература', 0.7988033294677734), ('литературный', 0.768280565738678), ('наследие', 0.7552648782730103), ('произведение', 0.7485151290893555), ('школа', 0.7300301790237427), ('наука', 0.7288336157798767), ('колледж', 0.7242863178253174), ('изобразительный', 0.7233369946479797), ('современный', 0.7229183912277222)]


3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия

In [10]:
max_item_len = 140

In [11]:
dict_idx = {}
def prepere_tfidf(data):
    X = []
    y = []

    for sentences in data:
        category, headline, text= sentences[0], sentences[1], sentences[2]

        category_idx = dict_idx.get(category, len(dict_idx))
        dict_idx[category] = category_idx
        
        word_idx = 0
        sentence_idx = 0
        pos_in_sent = 0

        x = []

        while word_idx < max_item_len:
            if word_idx < len(headline):
                x.append(headline[word_idx])
                word_idx += 1
            else:
                if len(text)!=0 and pos_in_sent < len(text[sentence_idx]):
                    x.append(text[sentence_idx][pos_in_sent])
                    word_idx += 1
                    pos_in_sent += 1
                elif sentence_idx < len(text) - 1:
                    sentence_idx += 1
                    pos_in_sent = 0
                else:
                    x.append("tmp")
                    word_idx += 1

        X.append(" ".join(x))
        y.append(category_idx)
    
    return X, y

x_train_idf, y_train = prepere_tfidf(data_train)
x_test_idf, y_test = prepere_tfidf(data_test)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
x_train_idf = tfidf.fit_transform(x_train_idf)
x_test_idf = tfidf.transform(x_test_idf)

In [13]:
x_train_idf.shape

(15000, 75116)

ЛОгистическая регрессия

In [14]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(n_jobs=-1)
clf.fit(x_train_idf, y_train)
preds = clf.predict(x_test_idf)
print("точность: " , (y_test == preds).mean())

точность:  0.8583333333333333


подбор гиперпараметров

In [19]:
from sklearn.model_selection import GridSearchCV

hyper = {'C' : [1, 4, 10, 100], 'solver': ['lbfgs']}
gd = GridSearchCV(estimator=
                LogisticRegression(multi_class='multinomial', random_state=17, n_jobs=4),
                param_grid=hyper)
gd.fit(x_train_idf, y_train)
print(gd.best_score_)
print(gd.best_estimator_)

0.8612666666666666
LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=4, penalty='l2',
                   random_state=17, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [21]:
clf_lr = gd.best_estimator_
clf_lr.fit(x_train_idf, y_train)
preds = clf_lr.predict(x_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

accuracy = 0.8743333333333333


наивный байесовский классификатор

In [15]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(x_train_idf, y_train)
preds = clf.predict(x_test_idf)
print("точность: " , (y_test == preds).mean())

точность:  0.798


In [15]:
(гиперпараметры отсутствуют)